In [1]:
#As always, we import everything
import pandas as pd
import os
import re
import hdf5_getters as getters
import requests
from bs4 import BeautifulSoup
import numpy as np
from collections import OrderedDict
from tqdm import tqdm
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords as stop_words
from textblob import Word
import pycountry
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from gensim import corpora, models
import matplotlib as plt
import json
import pickle
import lyricfetcher
import csv
import urllib
from urllib.request import urlopen, HTTPError
import langdetect

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 500

In [2]:
data_dir = '.' + '/data'

In [3]:
# start here

In [4]:
final_dataframe = pd.read_csv('data104863final_dataframe.csv')

final_dataframe.drop(final_dataframe[final_dataframe['Lyrics']==''].index, inplace=True)
final_dataframe.dropna(inplace=True)
slim_df = pd.read_csv('lyrics_slim_arthur.csv')
final_dataframe = pd.concat([slim_df, final_dataframe], ignore_index=True)
final_dataframe.drop('Unnamed: 0', axis =1, inplace=True)
final_dataframe.drop_duplicates(inplace = True)

In [5]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(sublinear_tf=True, min_df=2, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
# test_lyrics = tfidf.fit_transform(final_dataframe['Lyrics']).toarray()
# test_lyrics.shape
final_dataframe.shape

(51343, 3)

In [6]:
train_dataframe = pd.read_csv('train_lyrics.csv')

In [7]:
train_dataframe.drop(train_dataframe[train_dataframe['Lyrics']==''].index, inplace=True)
train_dataframe.dropna(inplace=True)
labels = train_dataframe.labels
col1 = ["Lyrics"]
train_dataframe = train_dataframe[col1]

In [8]:
train_dataframe.shape

(532, 1)

In [9]:
final_dataframe = final_dataframe[col1]

In [10]:
whole_corpus = pd.concat([train_dataframe, final_dataframe])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=2, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(whole_corpus['Lyrics']).toarray()

In [11]:
# free memory?
import gc

del train_dataframe
del final_dataframe

gc.collect()

0

In [12]:
features.shape

(51875, 442653)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

# since our training data is appended in the beginning of whole_corpus, take the first 531 to train the model
model = LinearSVC()
#X_train, X_test, 
#X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
model.fit(features[:532], labels[:532])




LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [14]:
features= features[50000:]
y_pred = model.predict(features)
print(len(y_pred))


1875


In [ ]:
np.savetxt("50000to_end.txt", y_pred, delimiter=",")